##Problem Statement
Suppose you are starting a company that grows and sells wild mushrooms. 
- Since not all mushrooms are edible, you'd like to be able to tell whether a given mushroom is edible or poisonous based on it's physical attributes
- You have some existing data that you can use for this task. 

Can you use the data to help you identify which mushrooms can be sold safely? 

Note: The dataset used is for illustrative purposes only. It is not meant to be a guide on identifying edible mushrooms.



<a name="3"></a>
## Dataset

You will start by loading the dataset for this task. The dataset you have collected is as follows:

|                                                     | Cap Color | Stalk Shape | Solitary | Edible |
|:---------------------------------------------------:|:---------:|:-----------:|:--------:|:------:|
| 00 |   Brown   |   Tapering  |    Yes   |    1   |
| 01 |   Brown   |  Enlarging  |    Yes   |    1   |
| 02 |   Brown   |  Enlarging  |    No    |    0   |
| 03 |   Brown   |  Enlarging  |    No    |    0   |
| 04 |   Brown   |   Tapering  |    Yes   |    1   |
| 05 |    Red    |   Tapering  |    Yes   |    0   |
| 06 |    Red    |  Enlarging  |    No    |    0   |
| 07 |   Brown   |  Enlarging  |    Yes   |    1   |
| 08 |    Red    |   Tapering  |    No    |    1   |
| 09 |   Brown   |  Enlarging  |    No    |    0   |


-  You have 10 examples of mushrooms. For each example, you have
    - Three features
        - Cap Color (`Brown` or `Red`),
        - Stalk Shape (`Tapering (as in \/)` or `Enlarging (as in /\)`), and
        - Solitary (`Yes` or `No`)
    - Label
        - Edible (`1` indicating yes or `0` indicating poisonous)

<a name="3.1"></a>
###One hot encoded dataset
For ease of implementation, we have one-hot encoded the features (turned them into 0 or 1 valued features)

|                                                    | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:--------------------------------------------------:|:---------:|:--------------------:|:--------:|:------:|
| 00 |     1     |           1          |     1    |    1   |
| 01 |     1     |           0          |     1    |    1   |
| 02 |     1     |           0          |     0    |    0   |
| 03 |     1     |           0          |     0    |    0   |
| 04 |     1     |           1          |     1    |    1   |
| 05 |     0     |           1          |     1    |    0   |
| 06 |     0     |           0          |     0    |    0   |
| 07 |     1     |           0          |     1    |    1   |
| 08 |     0     |           1          |     0    |    1   |
| 09 |     1     |           0          |     0    |    0   |


Therefore,
- `X_train` contains three features for each example 
    - Brown Color (A value of `1` indicates "Brown" cap color and `0` indicates "Red" cap color)
    - Tapering Shape (A value of `1` indicates "Tapering Stalk Shape" and `0` indicates "Enlarging" stalk shape)
    - Solitary  (A value of `1` indicates "Yes" and `0` indicates "No")

- `y_train` is whether the mushroom is edible 
    - `y = 1` indicates edible
    - `y = 0` indicates poisonous

In [1]:
## import nessasary libraries
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
## Define training sets x and y
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [3]:
# Calculate Entropy
def compute_entropy(y):
  '''
  y(ndarray): array that contains whether eatable mushroom or not in the considered node
  returns: a measurement of the impurity of that node
  '''
  y_positives = sum(y)
  if (len(y) != 0):
    y_ = y_positives / len(y)
    if (y_ == 0 or y_ == 1):
      return 0
    else:
      return -y_ * np.log2(y_) - (1 - y_) * np.log2(1 - y_)
  return 0

In [4]:
# Split the data
def split_data(X, y, node_indices, feature):
  '''
  returns: split the data in the current node left and right according to considered feature
  '''
  left_indices = []
  right_indices = []
  for indice in node_indices:
    if (X[indice][feature] == 1):
      left_indices.append(indice)
    else:
      right_indices.append(indice)

  return left_indices, right_indices

In [5]:
# Calculate information gain 
def compute_information_gain(X, y, node_indices, feature):
  '''
  X(ndarray): X training set
  y(ndarray): y training set || edible (`1`) or poisonous (`0`)
  node_iindices(ndarray): subset of X_train that contains only mushrooms related to the current node
  feature(int): feature to split on
  
  returns: information gain which means the difference between the impurity of the parent node and the average impurity of its left and right nodes
  '''
  left_indices, right_indices = split_data(X, y, node_indices, feature)

  w_left = len(left_indices) / len(node_indices)
  w_right = len(right_indices) / len(node_indices)

  y_node = y[node_indices]
  y_left = y[left_indices]
  y_right = y[right_indices]

  node_entropy = compute_entropy(y_node)
  weighted_entropy = w_left * compute_entropy(y_left) + w_right * compute_entropy(y_right)

  return node_entropy - weighted_entropy

In [6]:
## Checking the compute_information_gain function properly works or not
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)

info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377


In [7]:
def get_best_split(X, y, node_indices):
  '''
  X(ndarray): X training set
  y(ndarray): y training set || edible (`1`) or poisonous (`0`)
  node_iindices(ndarray): subset of X_train that contains only mushrooms related to the current node

  returns: what is the best feature to split on data in current node
  '''
  num_features = X.shape[1]
  best_feature = -1
  max_info_gain = 0

  for i in range(num_features):
    info_gain = compute_information_gain(X, y, node_indices, i)
    if (info_gain > max_info_gain):
      max_info_gain = info_gain
      best_feature = i
  
  return best_feature

In [8]:
## Checking get_best_split function properly works or not
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

Best feature to split on: 2


In [9]:
def build_tree(X, y, node_indices, max_depth, current_depth=0):
  '''
  X(ndarray): X training set
  y(ndarray): y training set || edible (`1`) or poisonous (`0`)
  node_iindices(ndarray): subset of X_train that contains only mushrooms related to the current node
  max_depth(int): maximum depth that stops spliting
  current(depth): current depth

  build the decision tree recursivey
  '''
  if (current_depth >= max_depth):
    return;
  
  best_feature = get_best_split(X, y, node_indices)
  print(f"depth {current_depth}-> feature: {best_feature}")

  left_indices, right_indices = split_data(X, y, node_indices, best_feature)
  print(f"  --left: " + str(left_indices))
  print(f"  --right: " + str(right_indices))
  build_tree(X, y, left_indices, max_depth, current_depth+1)
  build_tree(X, y, right_indices, max_depth, current_depth+1)

In [10]:
## Print the decision tree
build_tree(X_train, y_train, root_indices, max_depth=2, current_depth=0)

depth 0-> feature: 2
  --left: [0, 1, 4, 5, 7]
  --right: [2, 3, 6, 8, 9]
depth 1-> feature: 0
  --left: [0, 1, 4, 7]
  --right: [5]
depth 1-> feature: 1
  --left: [8]
  --right: [2, 3, 6, 9]
